In [1]:
import numpy as np 
import os
import pandas as pd 
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import shutil
tf.disable_eager_execution()
import tensorflow_hub as hub
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.preprocessing import image


# Dataset Notes
Georgia Tech face database (128Mb) contains images of 50 people taken in two 
or three sessions between 06/01/99 and 11/15/99 at the Center for Signal and 
Image Processing at  Georgia Institute of Technology. All people in the database 
are represented by 15 color JPEG images with cluttered background taken at resolution 
640x480 pixels. The average size of the faces in these images is 150x150 pixels. 
The images are stored in 50 directories s1, ..., s50. In each directory there 
are 15 images 01.jpg, ..., 15.jpg corresponding to one person in the database. 

Each image is manually labeled to determine the position of the face in the image.
The label files contain four integers that describe the coordinates of the face rectangles 
and a string (s1, ..., s50) indicating the identity of the face. Assuming that the upper 
left corner of the image has the coordinates (0,0), the numbers correspond to the 
x_left, y_top, x_right, y_bottom coordinates of the face rectangle. 

The label files are named as follows: 
lab001-lab015 correspond to files 01.jpg,...,15.jpg  in s01
lab021-lab035 correspond to files 01.jpg,...,15.jpg  in s02
lab041-lab055 correspond to files 01.jpg,...,15.jpg  in s03
lab061-lab075 correspond to files 01.jpg,...,15.jpg  in s04
....


In [2]:
def display_image_grid(image_paths, rows=4, cols=4):
    fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
    axes = axes.ravel()

    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')  # Turn off axis numbers and labels
        axes[i].set_title(img_path.split('/')[-1])  # Optional: Set titles as filenames

    plt.tight_layout()
    plt.show()

In [3]:
def pad_image(image_path, target_size=(299, 299)):
    with Image.open(image_path) as img:
        old_size = img.size

        # Calculate the new size, maintaining the aspect ratio
        ratio = float(target_size[0])/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])

        img = img.resize(new_size, Image.LANCZOS)

        # Calculate the deltas for padding
        delta_w = target_size[0] - new_size[0]
        delta_h = target_size[1] - new_size[1]

        # Create a new image with the target size and paste the resized image onto it
        new_img = Image.new("RGB", target_size)
        new_img.paste(img, ((target_size[0] - new_size[0]) // 2,
                            (target_size[1] - new_size[1]) // 2))

        # Edge padding
        left, top, right, bottom = ((target_size[0] - new_size[0]) // 2,
                                    (target_size[1] - new_size[1]) // 2,
                                    (target_size[0] - new_size[0] - (target_size[0] - new_size[0]) // 2),
                                    (target_size[1] - new_size[1] - (target_size[1] - new_size[1]) // 2))
        
        new_img = ImageOps.expand(new_img, border=(left, top, right, bottom), fill=None)

        return np.array(new_img)

In [4]:
def group_subjects():
    source_dir = '/kaggle/input/cs5567-final/GTdb_crop/cropped_faces'
    destination_dir = 'organized_faces'

    # Create destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        if filename.endswith('.jpg'):
            # Extract subject identifier from the filename
            subject_id = filename.split('_')[0]

            # Create a directory for the subject if it doesn't exist
            subject_dir = os.path.join(destination_dir, subject_id)
            if not os.path.exists(subject_dir):
                os.makedirs(subject_dir)

            # Move the file to the subject's directory
            shutil.copy(os.path.join(source_dir, filename), os.path.join(subject_dir, filename))

    print("Images have been sorted into subject folders.")

In [5]:
def split_data():
    data_dir = '/kaggle/working/organized_faces'
    base_dir = 'split_faces'
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')

    # Create base directories if they don't exist
    for directory in [train_dir, val_dir, test_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    # Function to copy files to the respective directories
    def copy_files(files, dest_dir):
        subject_dest_dir = os.path.join(dest_dir, subject_folder)
        if not os.path.exists(subject_dest_dir):
            os.makedirs(subject_dest_dir)
        for file in files:
            src_file_path = os.path.join(subject_path, file)
            dest_file_path = os.path.join(subject_dest_dir, file)
            shutil.copy(src_file_path, dest_file_path)

    for subject_folder in os.listdir(data_dir):
        subject_path = os.path.join(data_dir, subject_folder)
        images = os.listdir(subject_path)

        # Splitting the data into training, validation, and testing
        train_val_images, test_images = images[:10], images[10:]
        train_images, val_images = train_test_split(train_val_images, test_size=0.2, random_state = 42)  # 80-20 split

        # Copy files to respective directories
        copy_files(train_images, train_dir)
        copy_files(val_images, val_dir)
        copy_files(test_images, test_dir)

In [6]:
def augment_data(train_datagen):
    def resize_with_padding(img, target_size=(299, 299)):
        # Resize the image while maintaining aspect ratio
        ratio = min(target_size[0] / img.size[0], target_size[1] / img.size[1])
        new_size = (int(img.size[0] * ratio), int(img.size[1] * ratio))
        img = img.resize(new_size, Image.ANTIALIAS)

        # Convert to numpy array for ease of calculation
        img_array = np.array(img)

        # Calculate the average color of the leftmost and rightmost columns
        left_margin = img_array[:, 0, :]
        right_margin = img_array[:, -1, :]
        avg_color = np.mean(np.concatenate([left_margin, right_margin]), axis=0, dtype=int)

        # Create a new image with the average color of the margins
        new_img = Image.new("RGB", target_size, tuple(avg_color))
        top_left_x = (target_size[0] - new_size[0]) // 2
        top_left_y = (target_size[1] - new_size[1]) // 2
        new_img.paste(img, (top_left_x, top_left_y))

        return new_img
    np.random.seed(42)

    directories = {
        'train': '/kaggle/working/split_faces/train',
        'test': '/kaggle/working/split_faces/test',
        'val': '/kaggle/working/split_faces/val'
    }

    augmented_dir = 'augmented_faces'

    for dir_type, dir_path in directories.items():
        for subject in os.listdir(dir_path):
            subject_dir = os.path.join(dir_path, subject)
            augmented_subject_dir = os.path.join(augmented_dir, dir_type, subject)

            if not os.path.exists(augmented_subject_dir):
                os.makedirs(augmented_subject_dir)

            for img_file in os.listdir(subject_dir):
                img_path = os.path.join(subject_dir, img_file)
                img = Image.open(img_path)
                img = resize_with_padding(img, (299, 299))
                img_array = img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                file_name_without_ext = os.path.splitext(img_file)[0]

                if dir_type == 'train':
                    # Apply augmentation for training images
                    i = 0
                    for batch in train_datagen.flow(img_array, batch_size=1,
                                                    save_to_dir=augmented_subject_dir,
                                                    save_prefix=file_name_without_ext + '_',
                                                    save_format='jpg'):
                        i += 1
                        if i >= 5:
                            break
                else:
                    # For test and val, only resize and save the image
                    img.save(os.path.join(augmented_subject_dir, f"{file_name_without_ext}.jpg"))

In [7]:
# Open an image
image = Image.open('/kaggle/input/cs5567-final/GTdb_crop/cropped_faces/s01_01.jpg')

# Get image size
width, height = image.size

print(f"The image shape is: Width = {width}, Height = {height}")

# Note: PIL does not provide the number of channels directly, but you can get it like this:
channels = len(image.getbands())
print(f"Number of channels: {channels}")

The image shape is: Width = 181, Height = 241
Number of channels: 3


In [8]:
group_subjects()

Images have been sorted into subject folders.


image_dir = '/kaggle/input/cs5567-final/GTdb_crop/cropped_faces'
padded_dir = 'padded_faces'
label_dir = '/kaggle/input/cs5567-final/labels_gt/labels'

# Create padded image directory if it doesn't exist
if not os.path.exists(padded_dir):
    os.makedirs(padded_dir)

# Iterate through each image file in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):  # Assuming images are in .jpg format
        image_path = os.path.join(image_dir, filename)
        padded_image = pad_image(image_path)

        # Convert back to PIL image and save
        pil_image = Image.fromarray(padded_image)
        save_path = os.path.join(padded_dir, filename)
        pil_image.save(save_path)

In [9]:
split_data()

In [10]:
train_datagen = ImageDataGenerator(
    rotation_range=15,             # Random rotation between -15 to 15 degrees
    width_shift_range=0.05,         # Horizontal shift, here 10% of the total width
    height_shift_range=0.05,        # Vertical shift, here 10% of the total height
    brightness_range=(0.2, 0.2),         # No brightness adjustment
    shear_range=0.0,               # No shear transformation
    zoom_range=0.05,                # No zoom
    channel_shift_range=20,       # No channel shift
    fill_mode='nearest',           # Fill strategy for new pixels after rotation/shift
    cval=0.0,                      # Value used for fill_mode="constant"
    horizontal_flip=False,         # No horizontal flip
    vertical_flip=False,           # No vertical flip
    rescale=None,                  # No rescaling factor (e.g., rescale=1./255)
    preprocessing_function=None,   # No preprocessing function
    data_format=None,              # Default data format
    validation_split=0.0,          # No validation split
    interpolation_order=1,         # Default interpolation order
    dtype=None                     # Default data type
)


In [11]:
augment_data(train_datagen)

/tmp/ipykernel_19/2788992328.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(new_size, Image.ANTIALIAS)


# Open an image
image = Image.open('/kaggle/working/augmented_faces/train/s01/s01_13__0_5161.jpg')

# Get image size
width, height = image.size

print(f"The image shape is: Width = {width}, Height = {height}")

# Note: PIL does not provide the number of channels directly, but you can get it like this:
channels = len(image.getbands())
print(f"Number of channels: {channels}")

image_files = [
    "/kaggle/working/split_faces/train/s33/s33_08.jpg",
    "/kaggle/working/augmented_faces/train/s33/s33_08__0_5486.jpg",
    "/kaggle/working/augmented_faces/train/s33/s33_08__0_8666.jpg",
    "/kaggle/working/augmented_faces/train/s33/s33_08__0_8322.jpg",

    "/kaggle/working/split_faces/train/s40/s40_12.jpg",
    "/kaggle/working/augmented_faces/train/s40/s40_12__0_9347.jpg",
    "/kaggle/working/augmented_faces/train/s40/s40_12__0_5348.jpg",
    "/kaggle/working/augmented_faces/train/s40/s40_12__0_4849.jpg",

    "/kaggle/working/split_faces/train/s36/s36_12.jpg",
    "/kaggle/working/augmented_faces/train/s36/s36_12__0_3876.jpg",
    "/kaggle/working/augmented_faces/train/s36/s36_12__0_345.jpg",
    "/kaggle/working/augmented_faces/train/s36/s36_12__0_8393.jpg",
    
    "/kaggle/working/split_faces/train/s28/s28_08.jpg",
    "/kaggle/working/augmented_faces/train/s28/s28_08__0_9098.jpg",
    "/kaggle/working/augmented_faces/train/s28/s28_08__0_988.jpg",
    "/kaggle/working/augmented_faces/train/s28/s28_08__0_9437.jpg"
]

# Call the function to display the images
display_image_grid(image_files)